In [1]:
import numpy as np
import tensorflow as tf

# Global variables.
NUM_LABELS = 2    # The number of labels.

# flag_name is second_digit, half_ones, sum_of_digits
flag_name = 'half_ones'
flag_length = '7'
flag_epochs = 10
flag_hidden = 5

In [2]:
def extract_data(filename):
    # Arrays to hold the labels and feature vectors.
    labels = []
    fvecs = []

    # Iterate over the rows, splitting the label from the features. Convert labels
    # to integers and features to floats.
    f = open(filename, 'r')
    
    for line in f:
        row = line.strip('\n').split(",")
        labels.append(int(row[0]))
        fvecs.append([float(x) for x in row[1:]])
        #print("Vector length " + str(len([float(x) for x in row[1:]])))

    # Convert the array of float arrays into a numpy float matrix.
    fvecs_np = np.matrix(fvecs).astype(np.float32)

    # Convert the array of int labels into a numpy array.
    labels_np = np.array(labels).astype(dtype=np.uint8)

    # Convert the int numpy array into a one-hot matrix.
    labels_onehot = (np.arange(NUM_LABELS) == labels_np[:, None]).astype(np.float32)

    # Return a pair of the feature matrix and the one-hot label matrix.
    return fvecs_np,labels_onehot

In [3]:
def init_weights(shape, init_method='xavier', xavier_params = (None, None)):
    if init_method == 'zeros':
        return tf.Variable(tf.zeros(shape, dtype=tf.float32))
    elif init_method == 'uniform':
        return tf.Variable(tf.random_normal(shape, stddev=0.01, dtype=tf.float32))
    else: #xavier
        (fan_in, fan_out) = xavier_params
        low = -4*np.sqrt(6.0/(fan_in + fan_out)) # {sigmoid:4, tanh:1} 
        high = 4*np.sqrt(6.0/(fan_in + fan_out))
        return tf.Variable(tf.random_uniform(shape, minval=low, maxval=high, dtype=tf.float32))

In [6]:
# For tensorboard
sess = tf.InteractiveSession()
    
# Get the data.
train_data_filename = "data/outfile-length" + flag_length + "-train-" + flag_name + ".csv"
test_data_filename = "data/outfile-length" + flag_length + "-eval-" + flag_name + ".csv"

# Extract it into numpy arrays.
train_data,train_labels = extract_data(train_data_filename)
test_data, test_labels = extract_data(test_data_filename)

# Get the shape of the training data.
train_size,num_features = train_data.shape

# Get the number of epochs for training.
num_epochs = flag_epochs

# Get the size of layer one.
num_hidden = flag_hidden
 
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
x = tf.placeholder("float", shape=[None, num_features])
y_ = tf.placeholder("float", shape=[None, NUM_LABELS])
        
# For the test data, hold the entire dataset in one constant node.
test_data_node = tf.constant(test_data)

# Define and initialize the network.

# Initialize the hidden weights and biases.
w_hidden = init_weights(
    [num_features, num_hidden],
    'xavier',
    xavier_params=(num_features, num_hidden))

b_hidden = init_weights([1,num_hidden],'zeros')

# The hidden layer.
hidden = tf.nn.tanh(tf.matmul(x,w_hidden) + b_hidden)
# subst tf.nn.relu here for rectified linear units, the original is tanh
# if you use relu, you should probably switch out `xavier` below for `uniform`

# Initialize the output weights and biases.
w_out = init_weights(
    [num_hidden, NUM_LABELS],
    'xavier',
    xavier_params=(num_hidden, NUM_LABELS))
    
b_out = init_weights([1,NUM_LABELS],'zeros')

# The output layer.
y = tf.nn.softmax(tf.matmul(hidden, w_out) + b_out)
    
# Stuff for tensorboard
w_hist = tf.histogram_summary("weights", w_hidden)
b_hist = tf.histogram_summary("biases", b_hidden)
y_hist = tf.histogram_summary("y", y)
    
# Optimization.
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    
# Evaluation.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

accuracy_summary = tf.scalar_summary("accuracy", accuracy)
	
# More tensorboard stuff: merge all the summaries and write them out
merged = tf.merge_all_summaries()
summary_filename = "logs/" + flag_name + "-length" + flag_length
writer = tf.train.SummaryWriter(summary_filename, sess.graph)
    
# Run all the initializers to prepare the trainable parameters.
tf.initialize_all_variables().run()
    	
print('Initialized!')
print
print('Training.')
    	
# TODO: at the moment TensorBoard is disabled
# Iterate and train.
for step in range(num_epochs * train_size):
    if step % 10 == 0: # just test, no learning
        feed = {x: test_data, y_: test_labels}
        result=sess.run(accuracy,feed_dict=feed)
        #summary_str = result[0]
        #acc = result[0]
        #writer.add_summary(summary_str, step)
        print("Accuracy at step %s: %s" % (step, result))
    else: # actually do learning
        offset = step % train_size
        batch_data = train_data[offset:(offset + 1), :]
        batch_labels = train_labels[offset:(offset + 1)]
        feed={x: batch_data, y_: batch_labels}
        sess.run(train_step, feed_dict=feed)
    
print(accuracy.eval({x: test_data, y_: test_labels}))

Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x1183dfbe0>>
Traceback (most recent call last):
  File "//anaconda/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 171, in __del__
    self.close()
  File "//anaconda/lib/python3.5/site-packages/tensorflow/python/client/session.py", line 976, in close
    self._default_session.__exit__(None, None, None)
  File "//anaconda/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "//anaconda/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 3378, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


Initialized!
Training.
Accuracy at step 0: 0.721311
Accuracy at step 10: 0.786885
Accuracy at step 20: 0.983607
Accuracy at step 30: 0.983607
Accuracy at step 40: 0.983607
Accuracy at step 50: 0.983607
Accuracy at step 60: 0.983607
Accuracy at step 70: 0.983607
Accuracy at step 80: 0.983607
Accuracy at step 90: 0.983607
Accuracy at step 100: 0.983607
Accuracy at step 110: 0.983607
Accuracy at step 120: 0.983607
Accuracy at step 130: 0.983607
Accuracy at step 140: 0.983607
Accuracy at step 150: 0.983607
Accuracy at step 160: 0.983607
Accuracy at step 170: 0.983607
Accuracy at step 180: 0.983607
Accuracy at step 190: 0.983607
Accuracy at step 200: 0.983607
Accuracy at step 210: 0.983607
Accuracy at step 220: 0.983607
Accuracy at step 230: 0.983607
Accuracy at step 240: 0.983607
Accuracy at step 250: 0.983607
Accuracy at step 260: 0.983607
Accuracy at step 270: 0.983607
Accuracy at step 280: 0.983607
Accuracy at step 290: 0.983607
Accuracy at step 300: 0.983607
Accuracy at step 310: 0.983